## Import libraries

In [1]:
import numpy as np
import cv2 as cv
import csv
import time 
import tensorflow as tf
from tensorflow.python.keras.models import load_model

## Setting capture window parameters and loading labels

In [2]:
#Window Parameters.
frameWidth=640
frameHeight=480
brightness= 180

#Setting threshold limit(tunable).
threshold= 0.95

font= cv.FONT_HERSHEY_SIMPLEX

# Loading class labels into dictionary.
with open('labels.csv', mode='r') as infile:
    reader = csv.reader(infile)
    mydict = {rows[0]:rows[1] for rows in reader}      

## Loading our model

In [3]:
model=load_model("tsc_model.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Preprocessing of frame

In [4]:

def preprocess(img):
      
    img = cv.resize(img,(32,32))
    
    #Converting to grayscale.
    img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    img = cv.equalizeHist(img)
    
    #Normalize image.
    img=img/255   
    
    # Reshaping image to (1,32,32,1).       
    img=img[np.newaxis,:,:,np.newaxis]
    return img

## Importing OpenVINO libraries for optimization of model

In [5]:
import keras2onnx
import onnx
import onnxruntime

## Converting model to ONNX format

In [6]:
# Convert to onnx model.
onnx_model = keras2onnx.convert_keras(model, model.name)
temp_model_file = 'tsc.onnx'

#Saving the model.
keras2onnx.save_model(onnx_model, temp_model_file)
sess = onnxruntime.InferenceSession(temp_model_file)

The ONNX operator number change on the optimization: 36 -> 19


## Importing Inference Engine libraries

In [7]:
from openvino.inference_engine import IECore
from openvino.inference_engine import IENetwork

## Using optimized model from OpenVINO to generate output

In [12]:
#Loading model for generating inference
def load_to_IE(model):
    
    ie = IECore()
    net = ie.read_network(model=r"C:\Program Files (x86)\Intel\openvino_2021.2.185\deployment_tools\model_optimizer\tsc.xml", weights=r"C:\Program Files (x86)\Intel\openvino_2021.2.185\deployment_tools\model_optimizer\tsc.bin")
    exec_net = ie.load_network(network=net, device_name="CPU")

    return exec_net


def do_inference(exec_net, image):
    
    input_blob = next(iter(exec_net.inputs))
    
    return exec_net.infer({input_blob: image})


# load our model
tsc_net = load_to_IE("tsc_model.h5")


# We need dynamically generated key for fetching output tensor
tsc_outputs = list(tsc_net.outputs.keys())

# Taking video input from camera.
   
cap= cv.VideoCapture(0)

prev_frame_time = 0
new_frame_time = 0

cap.set(3,frameWidth)
cap.set(4,frameHeight)
cap.set(10,brightness)


while True:
    
    # If videocapture fails, exit else continue. 
    ret, image = cap.read()
    if not ret:
        break
    
    #Taking video frame and converting into numpy array.
    img = np.asarray(image)
    
    # Preprocessing frame by calling method.
    img = preprocess(img)
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
  
    # converting the fps into integer
    fps = int(fps)
    # Formatting for output display text.
    cv.putText(image, "CLASS: ",(20,35),font,0.75,(0,0,255),2,cv.LINE_AA)
    cv.putText(image, "PROBABILITY: ",(20,75),font,0.75,(255,0,0),2,cv.LINE_AA)
    cv.putText(image, "FPS: ", (20,115), font, 0.75, (0, 255, 0), 2, cv.LINE_AA)
  
    
    # Inference
    output = do_inference(tsc_net, image=img)
    
    # Storing label with maximum probability and probability score.
    classIndex = np.argmax(output[tsc_outputs[0]],axis=1)
    confidence = np.amax(output[tsc_outputs[0]])
    
    
    print(classIndex, confidence)
    
    # If probability score satisfies threshold limit, show output.
    if confidence>threshold:
        cv.putText(image, str(classIndex)+" "+mydict[str(max(classIndex))],(120,35),font,0.75,(0,0,255),2,cv.LINE_AA)
        cv.putText(image, str(round(confidence,2)),(180,75),font,0.75,(255,0,0),2,cv.LINE_AA)
        cv.putText(image, str(fps),(180,115),font,0.75,(0,255,0),2,cv.LINE_AA)

    cv.imshow("Result", image)
    
    # Setting wait time.
    k = cv.waitKey(1) & 0xFF == ord('q')
    if k==10:
        break

cap.release()
cap.destroyAllWindows()



c:\users\avirup\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 'inputs' property of ExecutableNetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoCPtr objects which can be accessed by 'input_info' property.
  del sys.path[0]


[12] 0.9927188
[12] 0.9957469
[12] 0.99429905
[12] 0.9871371
[12] 0.95549506
[12] 0.98335624
[12] 0.9851451
[12] 0.96183664
[12] 0.9802018
[12] 0.957496
[12] 0.9510844
[12] 0.96084124
[12] 0.94687885
[12] 0.97786045
[12] 0.97822577
[12] 0.97707736
[12] 0.95419234
[12] 0.82803357
[12] 0.7636338
[37] 0.9462188
[37] 0.7308511
[37] 0.78562987
[37] 0.9560381
[37] 0.9556645
[37] 0.73217916
[5] 0.6842458
[5] 0.35329866
[39] 0.9250924
[17] 0.5574955
[17] 0.23109615
[6] 0.24987102
[39] 0.52210397
[5] 0.4005384
[37] 0.977029
[37] 0.9594978
[37] 0.9953548
[37] 0.8855091
[12] 0.9852878
[37] 0.9974197
[37] 0.9950324
[37] 0.9475254
[37] 0.87161344
[37] 0.9683769
[12] 0.99268496
[37] 0.9960967
[37] 0.99853075
[37] 0.99224734
[37] 0.9496325
[37] 0.98853546
[37] 0.90025324
[37] 0.94246304
[37] 0.87005985
[37] 0.97214127
[37] 0.99067724
[37] 0.9260729
[12] 0.56436473
[12] 0.8228828
[12] 0.8879738
[12] 0.7828351
[37] 0.78458285
[26] 0.66710883
[37] 0.9926837
[37] 0.99926335
[37] 0.81597215
[37] 0.8998136

[37] 0.62065923
[39] 0.50201803
[9] 0.64082414
[38] 0.2605632
[37] 0.76879144
[39] 0.4839367
[39] 0.2888816
[38] 0.38276768
[18] 0.28968108
[38] 0.44781378
[38] 0.57959604
[37] 0.37215284
[38] 0.7030894
[38] 0.28695002
[39] 0.3516567
[28] 0.5333824
[38] 0.32735237
[39] 0.35930932
[39] 0.930261
[39] 0.68391186
[39] 0.5488398
[26] 0.39343193
[26] 0.68694377
[39] 0.8593804
[37] 0.47996372
[22] 0.35895282
[37] 0.3675711
[39] 0.42697042
[18] 0.42069945
[38] 0.46537653
[9] 0.34639853
[37] 0.5558568
[18] 0.34957525
[1] 0.44411474
[1] 0.37264642
[26] 0.74577236
[26] 0.5429543
[26] 0.5447102
[37] 0.79866874
[26] 0.2563602
[18] 0.58071965
[37] 0.7119624
[37] 0.43980685
[37] 0.69139445
[39] 0.5541336
[38] 0.50397223
[9] 0.6380039
[39] 0.7076194
[39] 0.6853293
[39] 0.9191847
[39] 0.9955114
[39] 0.50944585
[39] 0.8976064
[38] 0.56229836
[38] 0.83502173
[39] 0.85496175
[39] 0.91932756
[39] 0.6999509
[1] 0.3977888
[1] 0.80302614
[1] 0.7229947
[1] 0.7450466
[1] 0.7431059
[1] 0.61142176
[40] 0.7764708


[12] 0.9890336
[38] 0.53449094
[5] 0.6073858
[5] 0.63919836
[12] 0.22455028
[39] 0.52788335
[38] 0.44336593
[12] 0.2972834
[38] 0.39795065
[12] 0.6026765
[12] 0.46063545
[7] 0.32931447
[17] 0.4353207
[42] 0.3380952
[5] 0.26976478
[37] 0.9978782
[37] 0.98978686
[37] 0.99833703
[37] 0.9978847
[37] 0.9931062
[37] 0.77249235
[13] 0.6436537
[35] 0.4319677
[3] 0.32220423
[37] 0.84270287
[37] 0.64476836
[26] 0.32129416
[38] 0.9127719
[40] 0.29611647
[18] 0.9452077
[18] 0.43931988
[18] 0.5546885
[26] 0.5808692
[3] 0.76087445
[25] 0.9141363
[15] 0.34330687
[15] 0.46960303
[20] 0.5271981
[20] 0.3431953
[20] 0.6545041
[41] 0.94166785
[41] 0.43020698
[41] 0.88326883
[20] 0.37778407
[13] 0.5908057
[3] 0.46875072
[13] 0.31657144
[38] 0.936728
[5] 0.40820053
[11] 0.6748506
[13] 0.37350893
[20] 0.74374956
[26] 0.58595353
[18] 0.40302765
[23] 0.49910027
[23] 0.70020187
[26] 0.31585887
[23] 0.32343873
[23] 0.77004546
[20] 0.42246938
[20] 0.6139008
[28] 0.38365263
[28] 0.8631329
[28] 0.78765154
[28] 0.59

[23] 0.55370605
[40] 0.5911087
[12] 0.99513334
[29] 0.51268774
[38] 0.52765137
[40] 0.2881101
[19] 0.20319813
[38] 0.5208303
[38] 0.5245432
[38] 0.8110167
[40] 0.5323153
[12] 0.42084172
[38] 0.76633245
[38] 0.41369236
[40] 0.58552325
[40] 0.6419852
[12] 0.795041
[12] 0.6442829
[12] 0.9785346
[40] 0.8405052
[40] 0.3555593
[12] 0.5178397
[12] 0.8955138
[12] 0.69792634
[12] 0.8110676
[12] 0.69540745
[11] 0.2732276
[12] 0.38344982
[38] 0.52539396
[12] 0.9619681
[38] 0.7006903
[38] 0.97416013
[38] 0.88336146
[38] 0.91981936
[38] 0.85961777
[38] 0.93211406
[38] 0.97228515
[38] 0.95004934
[38] 0.4935348
[12] 0.88760346
[12] 0.9784918
[12] 0.98385817
[12] 0.98925054
[12] 0.81202817
[12] 0.92045176
[38] 0.95853275
[38] 0.9994085
[38] 0.9616796
[38] 0.9999753
[38] 0.98516965
[12] 0.8447216
[12] 0.8346878
[38] 0.8442934
[38] 0.9604389
[38] 0.98333156
[38] 0.9535505
[12] 0.67496353
[12] 0.9670589
[12] 0.99295264
[12] 0.99718493
[12] 0.75804347
[12] 0.67446774
[11] 0.7399338
[12] 0.99874073
[12] 0.

[15] 0.6221797
[15] 0.49442318
[20] 0.3578947
[15] 0.3591919
[38] 0.6156027
[20] 0.6226293
[38] 0.80080825
[26] 0.4003062
[26] 0.57517767
[30] 0.2837158
[29] 0.50835866
[25] 0.3030897
[29] 0.8114962
[29] 0.5446134
[29] 0.9996915
[29] 0.9277212
[8] 0.87820023
[29] 0.97343224
[29] 0.7796823
[29] 0.456604
[28] 0.35490757
[38] 0.9225061
[5] 0.38677898
[3] 0.48383778
[3] 0.4771024
[35] 0.2966414
[35] 0.48582968
[35] 0.5516213
[22] 0.6371484
[34] 0.9318545
[34] 0.4381646
[39] 0.46657988
[19] 0.35534778
[39] 0.9244447
[3] 0.74842435
[3] 0.60148704
[15] 0.608385
[3] 0.5515407
[3] 0.7722698
[3] 0.58694136
[3] 0.9467844
[15] 0.9594222
[3] 0.96343154
[15] 0.5259526
[3] 0.9243629
[3] 0.8448317
[3] 0.9986852
[3] 0.79025275
[3] 0.6354131
[3] 0.7116868
[15] 0.7589942
[13] 0.30788496
[3] 0.69141686
[9] 0.4326956
[3] 0.41225144
[3] 0.5499907
[3] 0.6459809
[35] 0.30703163
[15] 0.66413003
[15] 0.79313165
[9] 0.9104886
[15] 0.4899035
[15] 0.6148601
[26] 0.7127379
[26] 0.4512029
[9] 0.83482647
[9] 0.927457

[38] 0.98711944
[24] 0.4573299
[24] 0.97079414
[24] 0.5582895
[38] 0.8713744
[38] 0.92722875
[38] 0.9858372
[38] 0.9723159
[38] 0.83767253
[29] 0.5281868
[38] 0.891534
[38] 0.7641202
[38] 0.7406842
[38] 0.99952793
[38] 0.99996686
[38] 0.9754607
[26] 0.38533467
[21] 0.5303035
[26] 0.44944343
[26] 0.92967737
[26] 0.3985015
[26] 0.9470439
[20] 0.64059573
[2] 0.9723775
[4] 0.7779429
[38] 0.251685
[5] 0.7402248
[5] 0.91506666
[38] 0.22691819
[38] 0.8669349
[38] 0.830487
[2] 0.5391776
[26] 0.31216067
[38] 0.43648496
[38] 0.65713423
[38] 0.5792084
[38] 0.93090206
[38] 0.8536
[11] 0.6900321
[35] 0.60893023
[13] 0.9767527
[15] 0.19095676
[13] 0.6042801
[38] 0.7207525
[38] 0.5058294
[13] 0.75568515
[38] 0.9941552
[38] 0.99950397
[38] 0.99759054
[38] 0.9817152
[38] 0.7992969
[5] 0.43408006
[22] 0.52532583
[13] 0.35116643
[38] 0.98551303
[20] 0.6122612
[39] 0.6685028
[39] 0.8663458
[39] 0.6697373
[32] 0.6639856
[37] 0.67580795
[37] 0.5439002
[37] 0.2245612
[37] 0.25319454
[32] 0.76962954
[38] 0.38

KeyboardInterrupt: 